# Creating Personalized Text Chatbot

This jupyter notebook creates a Long short-term memory (LSTM) network for a ChatBot trained on my texts. The goal is to create a ChatBot that would replicate my behaviors, texting patterns, and colloqialisms. Our model is a seq2seq model that utilizes and an encoder and decoder to decipher text messages.

In [1]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing

In [2]:
print(tf.__version__)

2.11.0


Some preprocessing was done on my text dataset. I will include instructions on how to download your iMessage dataset below. For privacy reasons, I will not be sharing my "texts.txt" file. But the general look of it is as follows:
Mateen (me): Text
Responder: Response

https://www.switchingtomac.com/tutorials/how-to-download-your-imessage-chat-history/

Read in the text file and split the texts into questions and answers and then tag+tokenize them so the model is able to understand them.

In [4]:
from tensorflow.keras import preprocessing , utils
import os
import yaml

questions = []
answers = []
with open('texts.txt', 'r', encoding="utf-8") as f:
    for line in f:
        if line.startswith('Text'):
            questions.append(line[6:].rstrip('\n'))
        else:
            if line.startswith('Mateen'):
                answers.append(line[8:].rstrip('\n'))
                

answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
print(len(questions), len(answers))
questions = questions[:2000]
answers = answers[:2000]
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))


24054 24055
VOCAB SIZE : 2840


Tokenize our input and output training sets.

In [5]:
from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append( word )

def tokenize( sentences ):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub( '[^a-zA-Z]', ' ', sentence )
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append( tokens )
    return tokens_list , vocabulary

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )
print('done')

(2000, 174) 174
(2000, 63) 63
(2000, 63, 2840)
done


Create our model which uses embeddings to map the input tokens to continuous vectors before processing them through the LSTM layers.

In [6]:
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 174)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 63)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 174, 200)     568000      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 63, 200)      568000      ['input_2[0][0]']                
                                                                                              

In [7]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=150 ) 
model.save( 'model_small.h5' ) 

Epoch 1/150
40/40 [==============================] - 24s 483ms/step - loss: 6.8438
Epoch 2/150
40/40 [==============================] - 20s 490ms/step - loss: 5.4911
Epoch 3/150
40/40 [==============================] - 18s 457ms/step - loss: 5.2729
Epoch 4/150
40/40 [==============================] - 19s 467ms/step - loss: 5.0357
Epoch 5/150
40/40 [==============================] - 19s 473ms/step - loss: 4.8900
Epoch 6/150
40/40 [==============================] - 19s 485ms/step - loss: 4.8112
Epoch 7/150
40/40 [==============================] - 19s 478ms/step - loss: 4.7788
Epoch 8/150
40/40 [==============================] - 19s 476ms/step - loss: 4.7430
Epoch 9/150
40/40 [==============================] - 19s 475ms/step - loss: 4.7124
Epoch 10/150
40/40 [==============================] - 19s 479ms/step - loss: 4.6852
Epoch 11/150
40/40 [==============================] - 20s 489ms/step - loss: 4.6536
Epoch 12/150
40/40 [==============================] - 20s 487ms/step - loss: 4.6228
E

Take our model that we just created and create 2 models from it, one encoder and one decoder, that will be able to decipher our messages that we input.

In [8]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [9]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [10]:

enc_model , dec_model = make_inference_models()

while True:
    try:
        user_input = input( 'Enter question : ' )
        if user_input == 'q':
            print('Quitting...Goodbye')
            exit()
        states_values = enc_model.predict( str_to_tokens( user_input ) )
        empty_target_seq = np.zeros( ( 1 , 1 ) )
        empty_target_seq[0, 0] = tokenizer.word_index['start']
        stop_condition = False
        decoded_translation = ''
        while not stop_condition :
            dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
            sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
            sampled_word = None
            for word , index in tokenizer.word_index.items() :
                if sampled_word_index == index :
                    decoded_translation += ' {}'.format( word )
                    sampled_word = word

            if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
                stop_condition = True

            empty_target_seq = np.zeros( ( 1 , 1 ) )  
            empty_target_seq[ 0 , 0 ] = sampled_word_index
            states_values = [ h , c ] 

        print( decoded_translation )
    except KeyError:
        print("I am not sure")
        continue


Enter question : Hello
1/1 [==============================] - 0s 18ms/step
 sorry end
Enter question : Hi
1/1 [==============================] - 0s 18ms/step
 i’m down to the family end
Enter question : Whats up
I am not sure
Enter question : wyd
1/1 [==============================] - 0s 19ms/step
 wait i’m down for the night end
Enter question : whats the hw
I am not sure
Enter question : Bye
1/1 [==============================] - 0s 19ms/step
 yes end
Enter question : goodnight
1/1 [==============================] - 0s 19ms/step
 end
Enter question : gn
1/1 [==============================] - 0s 20ms/step
 lol end
Enter question : lol
1/1 [==============================] - 0s 19ms/step
 lol end
Enter question : lmao
1/1 [==============================] - 0s 19ms/step
 ahaha end
Enter question : How are you
1/1 [==============================] - 0s 20ms/step
 i’m not end
Enter question : Why
1/1 [==============================] - 0s 19ms/step
 ok end
Enter question : where
1/1 [=======

KeyboardInterrupt: Interrupted by user